# Laborator 8

## Modele de regresie

Selectati 4 seturi de date de la una din adresele de mai jos:
1. [https://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html](https://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html)
1. https://data.world/datasets/regression
1. https://homepages.inf.ed.ac.uk/rbf/IAPR/researchers/MLPAGES/mldat.htm
1. https://data.world/nrippner/ols-regression-challenge
1. https://www.interviewquery.com/p/regression-datasets-and-projects

Excludeti seturile de date: Machine CPU, Boston Housing, Wisconsin Breast Cancer, Communities and Crime.

Acolo unde este cazul, efectuati missing value imputation, de exemplu cu metodele indicate in Laboratorul 6. Metoda de completare va fi inclusa intr-un pipeline (a se vedea cursul 8). 

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - referinta [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru optimizarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



In [1]:
import pandas as pd
import numpy as np
from typing import List
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold,cross_validate, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import uniform
import os

!pip install prettytable --upgrade
from prettytable import PrettyTable

## 1. Set ailerons

In [2]:
names:List[str] = ['climbRate','Sgz','p','q','curPitch', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate',
                  'diffDiffClb','SeTime1','SeTime2','SeTime3', 'SeTime4', 'SeTime5', 'SeTime6', 'SeTime7',
                  'SeTime8','SeTime9','SeTime10','SeTime11','SeTime12', 'SeTime13', 'SeTime14', 'diffSeTime1', 'diffSeTime2',
                  'diffSeTime3','diffSeTime4','diffSeTime5','diffSeTime6','diffSeTime7', 'diffSeTime8', 'diffSeTime9', 
                   'diffSeTime10', 'diffSeTime11', 'diffSeTime12','diffSeTime13','diffSeTime14','alpha','Se', 'goal']
data_aileron:pd.DataFrame = pd.read_csv('data/ailerons.data', names=names)
data_aileron.info()

if data_aileron.isna().any().any():
    data_aileron=data_aileron.dropna()
data_aileron.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7154 entries, 0 to 7153
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   climbRate     7154 non-null   int64  
 1   Sgz           7154 non-null   int64  
 2   p             7154 non-null   float64
 3   q             7154 non-null   float64
 4   curPitch      7154 non-null   float64
 5   curRoll       7154 non-null   float64
 6   absRoll       7154 non-null   int64  
 7   diffClb       7154 non-null   int64  
 8   diffRollRate  7154 non-null   float64
 9   diffDiffClb   7154 non-null   float64
 10  SeTime1       7154 non-null   float64
 11  SeTime2       7154 non-null   float64
 12  SeTime3       7154 non-null   float64
 13  SeTime4       7154 non-null   float64
 14  SeTime5       7154 non-null   float64
 15  SeTime6       7154 non-null   float64
 16  SeTime7       7154 non-null   float64
 17  SeTime8       7154 non-null   float64
 18  SeTime9       7154 non-null 

,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,...,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se,goal
count,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,7154.000000,...,7154.000000,7154.000000,7.154000e+03,7154.000000,7.154000e+03,7154.000000,7.154000e+03,7154.000000,7154.000000,7154.000000
mean,-10.631255,-12.284037,0.008229,0.059817,0.630816,0.108876,-10.948420,-0.960442,-0.000944,-0.047987,...,0.000000,-0.000099,-2.795639e-07,-0.000105,-2.795639e-07,-0.000094,-2.795639e-07,0.625804,0.022064,-0.000868
std,259.485584,26.477064,0.306046,0.118000,0.316845,0.965375,4.074347,10.375730,0.013210,1.115519,...,0.000017,0.000621,2.896085e-05,0.000655,5.287676e-05,0.000614,1.671900e-05,0.203732,0.007207,0.000412
min,-975.000000,-95.000000,-1.070000,-0.460000,-0.360000,-3.100000,-23.000000,-41.000000,-0.059000,-7.200000,...,-0.001000,-0.005000,-2.000000e-03,-0.006000,-4.000000e-03,-0.005000,-1.000000e-03,0.400000,0.012000,-0.003500
25%,-177.000000,-29.000000,-0.210000,-0.020000,0.420000,-0.600000,-13.000000,-8.000000,-0.009000,-0.300000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.500000,0.018000,-0.001100
50%,-23.000000,-13.000000,0.020000,0.060000,0.610000,0.200000,-11.000000,-1.000000,-0.002000,-0.100000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.600000,0.020000,-0.000800
75%,160.000000,4.000000,0.240000,0.130000,0.800000,0.700000,-8.000000,6.000000,0.008000,0.200000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.600000,0.023000,-0.000600
max,977.000000,94.000000,0.990000,0.620000,2.580000,2.800000,-3.000000,45.000000,0.056000,7.800000,...,0.001000,0.003000,1.000000e-03,0.004000,2.000000e-03,0.007000,0.000000e+00,2.000000,0.073000,-0.000300


In [3]:
X_df=data_aileron.drop(columns=['goal'])
y_df=data_aileron['goal']

X_train, X_test, y_train, y_test = train_test_split(X_df,y_df, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=15/85, random_state=1)

scaler=StandardScaler()

columns=X_train.columns
X_train_index=X_train.index
X_val_index=X_val.index
X_test_index=X_test.index


X_train_norm=scaler.fit_transform(X_train)
X_val_norm=scaler.transform(X_val)
X_test_norm=scaler.transform(X_test)

X_train_norm=pd.DataFrame(X_train_norm,index=X_train_index,columns=columns)
X_val_norm=pd.DataFrame(X_val_norm,index=X_val_index,columns=columns)
X_test_norm=pd.DataFrame(X_test_norm,index=X_test_index,columns=columns)

ones_column_train=np.ones(len(X_train))
ones_column_val=np.ones(len(X_val))
ones_column_test=np.ones(len(X_test))
ones_column_train=pd.Series(ones_column_train,index=X_train_index)
ones_column_val=pd.Series(ones_column_val,index=X_val_index)
ones_column_test=pd.Series(ones_column_test,index=X_test_index)

X_train_norm_one=X_train_norm.copy()
X_val_norm_one=X_val_norm.copy()
X_test_norm_one=X_test_norm.copy()
X_train_norm_one.insert(0,'Ones',ones_column_train)
X_val_norm_one.insert(0,'Ones',ones_column_val)
X_test_norm_one.insert(0,'Ones',ones_column_test)

X_train_norm_one

,Ones,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,...,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
373,1.0,0.196179,-1.390304,0.301756,-1.196447,-0.347477,1.317203,0.479733,1.161589,-0.845591,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.428405
3748,1.0,0.095593,1.566044,1.291899,0.852033,0.034844,0.087838,0.726373,-0.966468,-0.921988,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,0.387334,0.288754
1355,1.0,1.093718,0.580595,0.466780,-0.342913,-1.112119,0.087838,-0.013546,-0.095899,-0.616403,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.571836
3123,1.0,0.393483,-0.859677,-1.579515,-0.257560,-0.379337,-0.014609,-0.013546,0.291020,-1.303968,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.141541
1818,1.0,-0.956694,-0.556462,0.829832,-0.940387,0.576466,0.497626,-0.013546,0.968129,-1.074780,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.284973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,1.0,1.190435,0.466889,-0.226320,-1.111094,-1.303280,2.034332,-0.260186,1.258319,0.147559,...,0.156914,0.0,1.790470,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.715268
1340,1.0,1.368395,1.945062,-1.117449,0.254560,-1.271420,0.600073,0.233093,-0.579548,-0.616403,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.428405
6209,1.0,-0.279672,0.315281,2.150022,-0.172207,-0.124456,-1.653762,-1.493384,1.064859,2.592236,...,0.156914,0.0,3.420153,0.00632,1.696543,0.014134,0.164306,0.014134,-0.625133,-0.571836
6912,1.0,0.853858,0.542693,-0.490358,-0.769680,-1.016539,-0.731738,-0.260186,0.968129,-0.310818,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,1.786945,0.014134,-0.625133,-0.715268


In [4]:
X_val_norm_one

,Ones,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,...,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
1327,1.0,-0.476975,-1.087088,0.697813,-1.879274,-0.188176,-1.961103,-0.506825,2.128887,2.515840,...,0.156914,0.0,-1.468895,0.00632,0.156552,0.014134,0.164306,0.014134,-1.131367,-0.858700
355,1.0,-1.413201,0.656398,-0.556368,0.254560,1.787150,-1.039080,-0.013546,-0.289359,0.300351,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,0.893568,1.005913
2283,1.0,0.142017,0.049968,1.489927,1.620213,0.257865,-0.936632,-1.740023,0.387750,0.071163,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,0.387334,0.719049
1424,1.0,-0.426682,-0.329051,0.103727,0.083853,0.002984,0.292732,-0.013546,-0.192629,-1.151176,...,-3.319502,0.0,0.160788,0.00632,-1.383440,0.014134,0.164306,0.014134,-0.625133,-0.428405
835,1.0,-0.074630,-1.390304,1.654951,0.425267,-0.220036,1.214756,-0.506825,0.581210,-1.227572,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.284973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978,1.0,-0.237116,0.770104,-0.358339,0.254560,-0.092596,1.419650,0.726373,-0.579548,-0.921988,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.428405
114,1.0,-0.682017,0.997515,0.169737,-0.342913,0.162285,1.419650,0.479733,0.291020,-0.616403,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.571836
5931,1.0,-2.113436,1.376534,1.522932,1.278800,1.468549,-0.117056,1.219652,-1.837036,0.071163,...,-3.319502,0.0,-4.728260,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.141541
4263,1.0,-2.148255,-1.087088,-0.358339,0.510620,1.468549,1.522097,-1.740023,0.871399,-1.074780,...,0.156914,0.0,-3.098578,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.141541


In [5]:
X_test_norm_one

,Ones,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,...,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
6788,1.0,0.513412,-0.215345,-0.325334,0.425267,1.213668,-0.424397,-0.260186,-0.192629,-0.540007,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,2.918503,2.727094
5825,1.0,0.993131,1.300730,0.895842,0.083853,-0.952819,-0.526844,1.219652,-0.676278,-0.463610,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.284973
3905,1.0,0.466988,0.239478,1.786970,0.510620,-0.347477,-0.526844,-0.753465,-0.192629,0.758728,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,0.001891
2333,1.0,-0.740047,0.808006,0.301756,2.729807,1.341108,-1.551315,-2.479942,-1.450117,-0.616403,...,0.156914,0.0,0.160788,0.00632,-6.003414,0.014134,0.164306,0.014134,1.399801,1.436208
1750,1.0,0.672029,-1.845126,-1.381487,-1.623214,2.010170,-0.629291,-0.260186,1.548508,0.223955,...,0.156914,0.0,-1.468895,0.00632,-1.383440,0.014134,3.409585,0.014134,4.437204,4.304844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,1.0,1.058899,0.239478,0.400770,-0.172207,-0.920959,-0.834185,0.973012,-0.289359,0.300351,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.284973
2192,1.0,1.101455,-0.177443,0.664808,-0.513620,-1.143979,0.292732,-0.013546,0.581210,-0.081630,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.625133,-0.571836
674,1.0,-0.291278,0.656398,0.895842,3.583340,3.666896,-0.117056,-0.753465,-2.320686,-1.456761,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,5.449671,5.595730
1723,1.0,1.890670,-1.541911,-0.424349,1.449507,-1.526300,1.931885,-1.246744,-1.353387,-3.366665,...,0.156914,0.0,0.160788,0.00632,0.156552,0.014134,0.164306,0.014134,-0.118900,-0.141541


# Models

In [6]:
# Definirea modelelor
ridge_model = Ridge()
lasso_model = Lasso()
elastic_net_model = ElasticNet()
decision_tree_model = DecisionTreeRegressor()
linear_regression_model= LinearRegression()

# Lista modelelor 
models = [ridge_model, lasso_model, elastic_net_model, decision_tree_model, linear_regression_model,]
# Numele modelelor
model_names = ['Ridge', 'Lasso', 'Elastic Net','Decision Tree', 'Linear Regression']

# Lista parametrilor de căutare pentru fiecare model
param_grids = [
    {'alpha': [0.1, 1.0, 10.0]}, # Ridge
    {'alpha': [0.1, 1.0, 10.0]}, # Lasso
    {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.1, 0.5, 0.9]}, # Elastic Net
    {'max_depth': [None, 5, 10]}, # Decision Tree Regressor
    {}, # Linear Regression
]
#Numele coloanelor pentru dataframe
column_names=['model_name','search_strategy','train_mean_absolute_error', 'train_mean_squared_error', 'train_median_absolute_error','test_mean_absolute_error', 'test_mean_squared_error', 'test_median_absolute_error','fit_time', 'score_time']

# Set: ailerons

In [7]:
# DataFrame-ul pentru rezultate
results_df_ailerons = pd.DataFrame({'index': []})

for model, model_name, param_grid in zip(models, model_names, param_grids):
    # GridSearchCV pentru fiecare model
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train_norm_one, y_train)
    
    # Alegerea celui mai bun model
    best_model = grid_search.best_estimator_
    
    # Cross-validate
    cv_results = cross_validate(best_model, X_train_norm_one, y_train, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], cv=5, return_train_score=True)
    
    # Calcul mediilor
    mean_train_mae = np.mean(cv_results['train_neg_mean_absolute_error'])
    mean_test_mae = np.mean(cv_results['test_neg_mean_absolute_error'])
    mean_train_mse = np.mean(cv_results['train_neg_mean_squared_error'])
    mean_test_mse = np.mean(cv_results['test_neg_mean_squared_error'])
    mean_train_median = np.mean(cv_results['train_neg_median_absolute_error'])
    mean_test_median = np.mean(cv_results['test_neg_median_absolute_error'])
    mean_train_fit_time = np.mean(cv_results['fit_time'])
    mean_test_score_time = np.mean(cv_results['score_time'])
    
    # Adăugarea rezultatelor la DataFrame
    results_df_ailerons[model_name] = ['GridSearchCV', mean_train_mae, mean_train_mse, mean_train_median, mean_test_mae, mean_test_mse, mean_test_median, mean_train_fit_time, mean_test_score_time]

# Transpunerea DataFrame-ului
results_df_ailerons = results_df_ailerons.T

# Setarea primei linii ca header
results_df_ailerons.columns = results_df_ailerons.iloc[0]

results_df_ailerons = results_df_ailerons[1:]

# Resetarea indexului
results_df_ailerons = results_df_ailerons.reset_index()

results_df_ailerons.columns = column_names

# Afișarea rezultatelor
results_df_ailerons.head()

,model_name,search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,fit_time,score_time
0,Ridge,GridSearchCV,-0.000127,-0.0,-0.000096,-0.000128,-0.0,-0.000097,0.011321,0.005827
1,Lasso,GridSearchCV,-0.000313,-0.0,-0.000267,-0.000313,-0.0,-0.000267,0.010409,0.0056
2,Elastic Net,GridSearchCV,-0.000313,-0.0,-0.000267,-0.000313,-0.0,-0.000267,0.010183,0.004523
3,Decision Tree,GridSearchCV,-0.000145,-0.0,-0.000111,-0.000155,-0.0,-0.000114,0.038089,0.004582
4,Linear Regression,GridSearchCV,-0.000127,-0.0,-0.000096,-0.000128,-0.0,-0.000097,0.010222,0.004779


## convert str to positive number

In [8]:
def convert_str_to_positive(value:str)->float:
    '''
    converteste valorile negative din coloana de tip str la numere positive 
    :param value(string): numarul care trebuie convertit    
    '''
    try:
        numeric_value = float(value)
        return abs(numeric_value)
    except ValueError:
        return value

In [9]:
# Aplicarea funcției pe toate valorile din DataFrame
results_df_ailerons = results_df_ailerons.applymap(convert_str_to_positive)

# verificare daca indecsii sunt unici
results_df_ailerons.index.is_unique

True

## Definirea regulilor de formatare pentru valorile minime si maxime

In [10]:
# Culorilor pentru valorile minime si maxime
min_color:str = 'yellow'
max_color:str = 'red'

# Selectarea coloanelor pentru aplicarea formatarei condiționate
columns_to_format = ['train_mean_absolute_error', 'train_mean_squared_error', 'test_mean_absolute_error',  'test_mean_squared_error', 'test_median_absolute_error', 'test_median_squared_error', 'fit_time', 'score_time']

In [11]:
# Aplicarea funcției de colorare pe DataFrame
results_df_ailerons.style.highlight_max(color='green').highlight_min(color='red')

# Coloanele pe care trebuie aplicată funcția de colorare  (toate coloanele, mai puțin primele două)
numeric_columns = results_df_ailerons.columns[2:]

df_styled_ailerons=results_df_ailerons.style.apply(lambda x: ['' for _ in x], subset=results_df_ailerons.columns[:2]) \
                          .highlight_max(color='lightgreen', subset=numeric_columns) \
                          .highlight_min(color='red', subset=numeric_columns)
df_styled_ailerons

,model_name,search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,fit_time,score_time
0,Ridge,GridSearchCV,0.000127,0.000000,0.000096,0.000128,0.000000,0.000097,0.011321,0.005827
1,Lasso,GridSearchCV,0.000313,0.000000,0.000267,0.000313,0.000000,0.000267,0.010409,0.005600
2,Elastic Net,GridSearchCV,0.000313,0.000000,0.000267,0.000313,0.000000,0.000267,0.010183,0.004523
3,Decision Tree,GridSearchCV,0.000145,0.000000,0.000111,0.000155,0.000000,0.000114,0.038089,0.004582
4,Linear Regression,GridSearchCV,0.000127,0.000000,0.000096,0.000128,0.000000,0.000097,0.010222,0.004779


## 2. Set abalone

In [12]:
names:List[str] = ['sex','length','diameter','height','whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']
data_abalone:pd.DataFrame = pd.read_csv('data/abalone.data',names=names)
    
data_abalone.info()
if data_abalone.isna().any().any():
    data_abalone = data_abalone.dropna()
data_abalone.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             4177 non-null   object 
 1   length          4177 non-null   float64
 2   diameter        4177 non-null   float64
 3   height          4177 non-null   float64
 4   whole_weight    4177 non-null   float64
 5   shucked_weight  4177 non-null   float64
 6   viscera_weight  4177 non-null   float64
 7   shell_weight    4177 non-null   float64
 8   rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,4177,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,NaN,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,NaN,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,NaN,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,NaN,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,NaN,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000


In [13]:
for label in "MFI":
    data_abalone[label] = data_abalone["sex"] == label
del data_abalone["sex"]

data_abalone.head()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings,M,F,I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,False,True


In [14]:
X = data_abalone.drop(columns=['rings'])
y = data_abalone['rings']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=15/85, random_state=1)

scaler=StandardScaler()

columns=X_train.columns
X_train_index=X_train.index
X_val_index=X_val.index
X_test_index=X_test.index


X_train_norm=scaler.fit_transform(X_train)
X_val_norm=scaler.transform(X_val)
X_test_norm=scaler.transform(X_test)

X_train_norm=pd.DataFrame(X_train_norm,index=X_train_index,columns=columns)
X_val_norm=pd.DataFrame(X_val_norm,index=X_val_index,columns=columns)
X_test_norm=pd.DataFrame(X_test_norm,index=X_test_index,columns=columns)

ones_column_train=np.ones(len(X_train))
ones_column_val=np.ones(len(X_val))
ones_column_test=np.ones(len(X_test))
ones_column_train=pd.Series(ones_column_train,index=X_train_index)
ones_column_val=pd.Series(ones_column_val,index=X_val_index)
ones_column_test=pd.Series(ones_column_test,index=X_test_index)

X_train_norm_one=X_train_norm.copy()
X_val_norm_one=X_val_norm.copy()
X_test_norm_one=X_test_norm.copy()
X_train_norm_one.insert(0,'Ones',ones_column_train)
X_val_norm_one.insert(0,'Ones',ones_column_val)
X_test_norm_one.insert(0,'Ones',ones_column_test)

X_train_norm_one

,Ones,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,M,F,I
3407,1.0,-1.463960,-1.456676,-1.394124,-1.305324,-1.262737,-1.232792,-1.339253,-0.758216,-0.671819,1.444503
3962,1.0,1.630997,1.677601,1.275664,2.672035,3.206103,1.832140,2.459896,-0.758216,1.488496,-0.692280
2626,1.0,-2.551377,-2.518286,-2.090590,-1.595731,-1.530373,-1.566235,-1.610621,-0.758216,-0.671819,1.444503
738,1.0,0.083519,0.009357,-0.117269,0.055325,-0.342880,-0.264438,0.567463,1.318885,-0.671819,-0.692280
2864,1.0,-1.798550,-1.810546,-1.626280,-1.442405,-1.399929,-1.415500,-1.446372,-0.758216,-0.671819,1.444503
...,...,...,...,...,...,...,...,...,...,...,...
1024,1.0,1.003641,0.919308,0.347042,0.783373,1.136987,0.498369,0.556751,-0.758216,1.488496,-0.692280
3403,1.0,-1.463960,-1.456676,-0.697658,-1.270800,-1.330208,-1.118599,-1.182146,-0.758216,-0.671819,1.444503
2749,1.0,-0.167424,-0.293961,-0.117269,-0.601645,-0.437340,-0.789724,-0.539433,-0.758216,-0.671819,1.444503
2983,1.0,-0.000129,0.009357,-0.117269,-0.089879,0.201387,0.151223,-0.453737,-0.758216,1.488496,-0.692280


In [15]:
# DataFrame-ul pentru rezultate
results_df_abalone = pd.DataFrame({'index': []})

for model, model_name, param_grid in zip(models, model_names, param_grids):
    # GridSearchCV pentru fiecare model
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train_norm_one, y_train)
    
    # Alegerea celui mai bun model
    best_model = grid_search.best_estimator_
    
    # Cross-validate
    cv_results = cross_validate(best_model, X_train_norm_one, y_train, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], cv=5, return_train_score=True)
    
    # Calcul mediilor
    mean_train_mae = np.mean(cv_results['train_neg_mean_absolute_error'])
    mean_test_mae = np.mean(cv_results['test_neg_mean_absolute_error'])
    mean_train_mse = np.mean(cv_results['train_neg_mean_squared_error'])
    mean_test_mse = np.mean(cv_results['test_neg_mean_squared_error'])
    mean_train_median = np.mean(cv_results['train_neg_median_absolute_error'])
    mean_test_median = np.mean(cv_results['test_neg_median_absolute_error'])
    mean_train_fit_time = np.mean(cv_results['fit_time'])
    mean_test_score_time = np.mean(cv_results['score_time'])
    
    # Adăugarea rezultatelor la DataFrame
    results_df_abalone[model_name] = ['GridSearchCV', mean_train_mae, mean_train_mse, mean_train_median, mean_test_mae, mean_test_mse, mean_test_median, mean_train_fit_time, mean_test_score_time]

# Transpunerea DataFrame-ului
results_df_abalone = results_df_abalone.T

# Setarea primei linii ca header
results_df_abalone.columns = results_df_abalone.iloc[0]

results_df_abalone = results_df_abalone[1:]

# Resetarea indexului
results_df_abalone = results_df_abalone.reset_index()

results_df_abalone.columns = column_names

# Afișarea rezultatelor
results_df_abalone.head()

,model_name,search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,fit_time,score_time
0,Ridge,GridSearchCV,-1.574891,-4.81122,-1.15044,-1.583839,-4.99095,-1.156827,0.004043,0.002343
1,Lasso,GridSearchCV,-1.631967,-5.200067,-1.239472,-1.639172,-5.327815,-1.237649,0.0026,0.0022
2,Elastic Net,GridSearchCV,-1.633478,-5.214845,-1.242264,-1.640372,-5.341938,-1.241675,0.002801,0.002
3,Decision Tree,GridSearchCV,-1.487218,-4.492605,-1.048981,-1.623234,-5.365363,-1.098127,0.010597,0.002205
4,Linear Regression,GridSearchCV,-1.5749,-4.810657,-1.150879,-1.583901,-4.990843,-1.157513,0.001601,0.002


In [16]:
results_df_abalone = results_df_abalone.applymap(convert_str_to_positive)

results_df_abalone.index.is_unique

True

In [17]:
print(type(results_df_abalone))

<class 'pandas.core.frame.DataFrame'>


In [18]:
results_df_abalone.columns

Index(['model_name', 'search_strategy', 'train_mean_absolute_error',
       'train_mean_squared_error', 'train_median_absolute_error',
       'test_mean_absolute_error', 'test_mean_squared_error',
       'test_median_absolute_error', 'fit_time', 'score_time'],
      dtype='object')

In [19]:
#verificarea dacă există coloane duplicate
print (results_df_abalone.columns[results_df_abalone.columns.duplicated(keep=False)])

Index([], dtype='object')


In [20]:
results_df_abalone.style.highlight_max(color='green').highlight_min(color='red')

numeric_columns = results_df_abalone.columns[2:]

df_styled_abalone=results_df_abalone.style.apply(lambda x: ['' for _ in x], subset=results_df_abalone.columns[:2]) \
                          .highlight_max(color='lightgreen', subset=numeric_columns) \
                          .highlight_min(color='red', subset=numeric_columns)
df_styled_abalone

,model_name,search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,fit_time,score_time
0,Ridge,GridSearchCV,1.574891,4.811220,1.150440,1.583839,4.990950,1.156827,0.004043,0.002343
1,Lasso,GridSearchCV,1.631967,5.200067,1.239472,1.639172,5.327815,1.237649,0.002600,0.002200
2,Elastic Net,GridSearchCV,1.633478,5.214845,1.242264,1.640372,5.341938,1.241675,0.002801,0.002000
3,Decision Tree,GridSearchCV,1.487218,4.492605,1.048981,1.623234,5.365363,1.098127,0.010597,0.002205
4,Linear Regression,GridSearchCV,1.574900,4.810657,1.150879,1.583901,4.990843,1.157513,0.001601,0.002000


## 3.

## 4. HTML Reports

In [21]:
from nbconvert import HTMLExporter
from nbformat import read
from IPython.display import HTML
from jinja2 import Template

import webbrowser

In [22]:
# template HTML
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>{{title}}</title>
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <h1>{{dataset_name}}</h1>
    <h2>{{ dataset }}</h2>
    {{ df_html }}
</body>
</html>
"""

## data Report

In [23]:
# Compilarea tamplet-ului
template = Template(html_template)

html_ailerons = template.render(title='Ailerons',dataset_name='Ailerons data:',dataset=df_styled_ailerons, df_html=df_styled_ailerons.to_html(classes='table table-striped'))

# Salvare HTML într-un fișier
with open("raport_ailerons.html", "w") as f:
    f.write(html_ailerons)

# Deschiderea fișierului HTML în browser-ul implicit
webbrowser.open("raport_ailerons.html")

True

## Abalone data Report 

In [24]:
html_abalone = template.render(title='Abalone',dataset_name='Abalone data:',dataset=df_styled_abalone, df_html=df_styled_abalone.to_html(classes='table table-striped'))

with open("raport_abalone.html", "w") as f:
    f.write(html_abalone)

webbrowser.open("raport_abalone.html")

True

## Precizari

* Se va face o sectiune separata in cadrul notebook-ului pentru fiecare sursa de date. 
* Asigurati-va ca includeti fisierele de date folosite in arhiva predata
* Pot fi folosite aceleasi modele de regresie pentru toate seturile de date.
* Depunerea se face in Tema 6 pe elearning, pana cel tarziu 20 decembrie 2023 ora 23.
* Specificatiile legate de depunere, numele fisierelor etc. sunt aceleasi ca la temele precedente. 